In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import statsmodels.stats.multitest
import operator
import cptac.utils as u

In [62]:
sup18 = pd.read_excel(r"E:\Users\brittany henderson\Desktop\PIK3CA\nature18003-s2\CPTAC_BC_SupplementaryTable18.xlsx")
sup18["phospho_site"] =  sup18["Gene Name"] + "_" + sup18["Phosphosite"] + "_" + sup18["Description"]
sup18 = sup18[["Gene Name","FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)","phospho_site"]]
sup18 = sup18.rename(columns = {"FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)" : "Significant", "Gene Name" : "Gene_Name"})

KeyError: 'Description'

In [ ]:
sig_sup18 = sup18.loc[sup18['Significant'] == True]
sig_sup18

In [25]:
sig_g = sig_sup18.loc[sig_sup18["Gene_Name"] == "MAPT"]
sig_g

,Gene_Name,Significant,phospho_site
9,MAPT,True,MAPT_S726s T731t
10,MAPT,True,MAPT_S411s S437s
19,MAPT,True,MAPT_S531s
23,MAPT,True,MAPT_T111t
24,MAPT,True,MAPT_S531s T534t
25,MAPT,True,MAPT_S733s


In [117]:

#sup18_T = sup18.loc[sup18["Significant"] == True]
#cols_62 = sup18_T.phospho_site.to_list()
#cols_62

# Get phosphoproteomic data from paper

In [59]:
s4 = pd.read_excel(r"E:\Users\brittany henderson\Desktop\PIK3CA\nature18003-s2\CPTAC_BC_SupplementaryTable04.xlsx")
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,27]
s4 = s4.drop(s4.columns[cols],axis=1)

In [42]:
genes = ['MAPT']
gene = df[df['geneName'].isin(genes)]
gene

,variableSites,geneName,AO-A12D.01TCGA,C8-A131.01TCGA,AO-A12B.01TCGA,E2-A10A.02TCGA,BH-A18Q.02TCGA,C8-A130.02TCGA,C8-A138.03TCGA,E2-A154.03TCGA,...,AO-A12B.34TCGA,A2-A0SW.35TCGA,AO-A0JL.35TCGA,BH-A0BV.35TCGA,A2-A0YM.36TCGA,BH-A0C7.36TCGA,A2-A0SX.36TCGA,263d3f-I.CPTAC,blcdb9-I.CPTAC,c4155b-C.CPTAC
7092,T111t,MAPT,-1.007736,-2.616275,-1.506033,-0.446058,-2.700136,1.282824,NaN,NaN,...,-2.098282,-0.699004,-0.974505,1.755167,-1.136713,0.085801,-1.563016,NaN,NaN,NaN
7093,S68s,MAPT,-0.465045,-0.682875,-0.453999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7094,S84s,MAPT,-1.885555,-5.227752,1.022355,-1.952610,-3.770226,0.145665,-4.022604,0.903824,...,1.017096,-0.602101,-2.417872,-0.365599,-2.089064,2.085097,-2.899980,0.254949,-3.432338,-0.643437
7095,T82t S84s,MAPT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7096,S55s,MAPT,-2.019606,-5.139526,-1.455185,-4.171321,-5.354122,-0.431663,-4.834190,0.711850,...,-1.437658,-0.482520,-2.156174,-0.972964,-2.861318,-0.550641,-2.627784,-0.681110,-4.494260,-0.800850


In [41]:
num = df.geneName.value_counts().to_frame()
n = num.reset_index()
n[n['index'].str.contains('MAPT', regex = True,  na=False)]


,index,geneName
2759,MAPT,5


In [50]:
mpt = sup18[sup18['Gene_Name'].str.contains('MAPT', regex = True,  na=False)]
len(mpt)

39

In [6]:

df["phospho_site"] =  df["geneName"] + "_" + df["variableSites"]
#df = df[pd.notnull(df['phospho_site'])]
df = df.drop(["variableSites","geneName"], axis = 1)
#df = df[df['phoshpho_site'].isin(cols_62)]
#df1 = df.set_index("phospho_site")
df1.columns = df1.columns.str.replace(r'...TCGA', "", regex = True)
df1

KeyError: 'geneName'

AttributeError: 'Int64Index' object has no attribute 'contains'

In [111]:
df2 = df1.T
df2.index.names = ['Sample ID']
df2 = df2.dropna(how = "all", axis = 1)

df2.columns = df2.columns.str.replace(r' ', "_", regex = True)



# Get mutation data

In [29]:
mutations = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable01.csv")
mutations = mutations.set_index("Sample ID")
mutations = mutations[["PIK3CA Mutation"]]
mutations = mutations.rename(columns = {"PIK3CA Mutation" : "PIK3CA_Mutation"})
mutations =  mutations.replace(np.nan, 'Wildtype', regex=True)
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Missense_Mutation|Somatic|p.H1047R
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Missense_Mutation|Somatic|p.E545K
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [30]:
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E542K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('E545K')] = "Hotspot"
mutations.loc[mutations.PIK3CA_Mutation.str.contains('H1047R')] = "Hotspot"
mutations

,PIK3CA_Mutation
Sample ID,
A2-A0CM,Wildtype
A2-A0D2,Wildtype
A2-A0EQ,Hotspot
A2-A0EV,In_Frame_Ins|Somatic|p.E469in_frame_insDK
A2-A0EX,Hotspot
...,...
C8-A12W,Wildtype
D8-A13Y,Wildtype
E2-A10A,Wildtype


In [31]:
merged = pd.merge(df2, mutations, on = "Sample ID", how = "outer")
merged

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,BST1_S40s_,WNT9B_T234t_,C4ORF39_S92s_T107t_,AKD1_T1425t_,ZNF329_T359t_,GPR78_T210t_,SLC27A6_S217s_T218t_T224t_,PIGL_S250s_,RS1_Y165y_Y166y_T170t_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,NaN,-7.085443,-3.870377,0.014945,-0.698553,NaN,NaN,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,NaN,-0.410227,2.292365,-5.464890,-1.487679,NaN,NaN,NaN,NaN,Wildtype
C8-A131,-2.565861,-1.726458,NaN,-0.370305,-2.023904,0.637987,0.587573,0.693443,1.003493,-0.403074,...,NaN,NaN,NaN,1.676528,-3.110339,2.483162,-1.693688,NaN,NaN,Missense_Mutation|Somatic|p.R108H
C8-A131,-1.847555,-1.732055,NaN,-0.080658,-2.201425,NaN,NaN,NaN,NaN,-1.407674,...,NaN,NaN,NaN,NaN,NaN,2.983764,NaN,NaN,NaN,Missense_Mutation|Somatic|p.R108H
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,NaN,-0.213785,-2.580861,-0.324249,-0.175210,NaN,NaN,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BH-A0C7,1.728556,2.213385,-0.342381,NaN,2.001793,1.182082,0.239081,NaN,0.925506,1.260388,...,NaN,-0.129123,NaN,NaN,-2.434978,NaN,NaN,NaN,NaN,Wildtype
A2-A0SX,1.092898,0.444430,1.767384,NaN,-0.352144,0.488462,1.204979,NaN,1.345080,0.780673,...,NaN,-2.493687,NaN,NaN,-2.523709,NaN,NaN,NaN,NaN,Wildtype
263d3f-I.CPTAC,-1.153228,-0.331621,NaN,NaN,NaN,NaN,NaN,-0.603446,0.103708,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
wt_hotspot = merged.loc[(merged['PIK3CA_Mutation'] == "Wildtype") |
                (merged['PIK3CA_Mutation'] == "Hotspot")]

In [48]:
#wt_hotspot = wt_hotspot.dropna(how = "all")
#wt_hotspot = wt_hotspot.dropna(thresh= 20, axis = 1)
wt_hotspot

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,TICAM1_S223s_,FLJ37543_T38t_T42t_,COMMD6_S32s_,AFMID_S46s_,HOXA13_M1m_T2t_,AKD1_T1425t_,ZNF329_T359t_,PIGL_S250s_,RS1_Y165y_Y166y_T170t_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,NaN,NaN,NaN,NaN,NaN,-7.085443,-3.870377,NaN,NaN,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,NaN,NaN,NaN,NaN,-0.553622,-5.464890,-1.487679,NaN,NaN,Wildtype
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,NaN,NaN,NaN,NaN,NaN,-0.213785,-2.580861,NaN,NaN,Wildtype
AO-A12B,0.805357,-0.124880,0.880172,-0.795385,-0.915370,0.455283,NaN,NaN,-0.582235,0.851940,...,NaN,0.682549,NaN,NaN,NaN,NaN,-1.801849,NaN,NaN,Wildtype
E2-A10A,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,-1.209202,NaN,-1.268412,NaN,NaN,NaN,-0.449347,NaN,NaN,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AO-A0JL,1.338505,0.495032,0.364182,1.628386,3.490470,NaN,NaN,NaN,0.078327,0.813096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.528098,Wildtype
BH-A0BV,3.450978,0.776968,1.626879,1.382329,2.797509,NaN,NaN,NaN,0.289874,4.922284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.247333,Hotspot
A2-A0YM,0.200324,1.017243,1.985442,NaN,0.485597,1.053262,2.380214,NaN,1.344298,1.975357,...,0.345842,NaN,NaN,NaN,NaN,NaN,-1.485380,NaN,NaN,Wildtype


In [49]:


col_names = pd.Series(wt_hotspot.columns[:])
for dup in col_names[col_names.duplicated()].unique(): 
    col_names[col_names[col_names == dup].index.values.tolist()] = [dup + 'isoform_' + str(i) if i != 0 else dup for i in range(sum(col_names == dup))]
wt_hotspot.columns = col_names # rename the columns with the cols list


In [50]:
len(col_names)

32349

In [51]:
#wrap_ttest(wt_hotspot, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=False, pval_return_corrected=False)

In [52]:
subset_test = wt_hotspot.iloc[:, list(range(10000)) + [-1]]

subset_test = subset_test.dropna(thresh= 10, axis = 1)
subset_test

,AHNAK_S1007s_,AHNAK_S1010s_,AHNAK_S1023s_,AHNAK_S1042s_,AHNAK_S1068s_,AHNAK_S106s_,AHNAK_S106s_S109s_,AHNAK_S1088s_,AHNAK_S110s_,AHNAK_S1123s_,...,AKNA_S1043s_,AKNA_S1102s_,AKNA_S1161s_,AKNA_S1175s_,AKNA_S1228s_,AKNA_S1387s_,AKNA_S1422s_,AKNA_S192s_,AKNA_S272s_,PIK3CA_Mutation
Sample ID,,,,,,,,,,,,,,,,,,,,,
AO-A12D,-0.919089,-1.271514,NaN,-2.298519,-0.802335,-0.090999,-0.393695,0.378180,-0.374236,-1.777449,...,0.235481,NaN,-1.364485,NaN,-0.090999,-0.523422,-0.434775,NaN,0.038728,Wildtype
AO-A12D,0.029915,-1.190931,NaN,-2.049308,-1.141141,0.159369,-0.404252,0.631376,0.691124,-1.756544,...,-0.724899,NaN,-1.378141,-0.019875,-0.446075,-0.822487,-1.270595,-0.201110,NaN,Wildtype
AO-A12B,0.177221,-0.320742,NaN,-1.169382,-1.223738,0.059744,0.654143,0.028183,0.440229,1.103011,...,-0.033186,NaN,-1.355242,NaN,0.441983,-0.615311,-0.801171,NaN,-0.515368,Wildtype
AO-A12B,0.805357,-0.124880,0.880172,-0.795385,-0.915370,0.455283,NaN,NaN,-0.582235,0.851940,...,NaN,NaN,-0.977480,-0.412844,-0.815147,NaN,0.722074,0.064273,NaN,Wildtype
E2-A10A,0.203602,-0.523359,NaN,0.330244,-0.465794,0.540767,NaN,-4.867033,0.386164,0.754579,...,-1.913137,NaN,-6.684435,1.121349,-3.666396,0.001303,-4.080862,NaN,-2.526613,Wildtype
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AO-A0JL,1.338505,0.495032,0.364182,1.628386,3.490470,NaN,NaN,NaN,0.078327,0.813096,...,NaN,0.251451,NaN,NaN,-0.378638,-0.449095,NaN,NaN,NaN,Wildtype
BH-A0BV,3.450978,0.776968,1.626879,1.382329,2.797509,NaN,NaN,NaN,0.289874,4.922284,...,NaN,0.528410,NaN,NaN,1.464514,0.931315,NaN,NaN,NaN,Hotspot
A2-A0YM,0.200324,1.017243,1.985442,NaN,0.485597,1.053262,2.380214,NaN,1.344298,1.975357,...,NaN,0.465427,NaN,NaN,NaN,-0.125290,NaN,NaN,NaN,Wildtype


In [53]:
cols = list(subset_test.columns[:-1])
cols

['AHNAK_S1007s_',
 'AHNAK_S1010s_',
 'AHNAK_S1023s_',
 'AHNAK_S1042s_',
 'AHNAK_S1068s_',
 'AHNAK_S106s_',
 'AHNAK_S106s_S109s_',
 'AHNAK_S1088s_',
 'AHNAK_S110s_',
 'AHNAK_S1123s_',
 'AHNAK_S1135s_',
 'AHNAK_S1138s_',
 'AHNAK_S1158s_',
 'AHNAK_S115s_',
 'AHNAK_S1170s_',
 'AHNAK_S1196s_isoform_1',
 'AHNAK_S1216s_',
 'AHNAK_S1263s_',
 'AHNAK_S1286s_',
 'AHNAK_S1298s_',
 'AHNAK_S1324s_',
 'AHNAK_S1344s_',
 'AHNAK_S135s_',
 'AHNAK_S1367s_',
 'AHNAK_S1391s_',
 'AHNAK_M1441m_S1452s_',
 'AHNAK_S1452s_',
 'AHNAK_S1519s_',
 'AHNAK_S1542s_',
 'AHNAK_S1571s_M1572m_',
 'AHNAK_S1580s_',
 'AHNAK_S1600s_',
 'AHNAK_S1654s_',
 'AHNAK_S1674s_',
 'AHNAK_S1747s_',
 'AHNAK_S177s_',
 'AHNAK_S1782s_',
 'AHNAK_S1802s_',
 'AHNAK_S1876s_',
 'AHNAK_S1888s_',
 'AHNAK_S201s_',
 'AHNAK_S20s_',
 'AHNAK_S210s_',
 'AHNAK_S210s_S212s_S216s_',
 'AHNAK_S210s_S216s_',
 'AHNAK_S2138s_',
 'AHNAK_S216s_',
 'AHNAK_S216s_T218t_',
 'AHNAK_S2185s_',
 'AHNAK_S2252s_',
 'AHNAK_S2287s_',
 'AHNAK_S2313s_',
 'AHNAK_S232s_',
 'AHNAK_

In [54]:
u.wrap_ttest(subset_test, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=False)

,Comparison,P_Value
0,ILF3_S792s_,0.000010
1,MYH14_S220s_,0.000205
2,NUMA1_S200s_,0.000252
3,MAPT_T111t_,0.000444
4,NCOR1_T1555t_,0.000675
...,...,...
9595,ZNF318_S79s_S81s_,0.999248
9596,PARD3B_S1122s_,0.999392
9597,TENC1_S842s_S845s_,0.999601
9598,SYNPO2_S1181s_,0.999631


In [55]:
u.wrap_ttest(subset_test, "PIK3CA_Mutation", comparison_columns = cols , correction_method="fdr_bh", return_all=True, pval_return_corrected=True)

,Comparison,P_Value
0,ILF3_S792s_,0.092006
1,ARHGEF11_S556s_,0.798795
2,ACIN1_S216s_isoform_2,0.798795
3,DLG5_S1252s_,0.798795
4,ACIN1_S240s_S243s_,0.798795
...,...,...
9595,ZNF318_S79s_S81s_,0.999664
9596,PARD3B_S1122s_,0.999705
9597,SYNPO2_S1181s_,0.999735
9598,TENC1_S842s_S845s_,0.999735


In [61]:

sup18

,FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors),Gene Name,Phosphosite,RefSeq_psite_#sites_#localized_1staa_lastaa,RefSeq,Kinase,Kinase_substrate relationships in marker set (PhosphositePlus database; kinases in PIK3CA pathway),SAM D-score PIK3CA mutated vs not mutated across all tumors,Fold Change Luminal PIK3CA mutation markers (linear; mutated/wt),Fold Change all tumors PIK3CA mutation markers (linear; mutated/wt),Fold Change all tumors PIK3CA missense mutation in kinase domain markers (linear; mutated/wt),Fold Change all tumors PIK3CA missense mutation in helical domain markers (linear; mutated/wt),"q-value(%) SAM, Luminal PIK3CA mutation markers","q-value(%) SAM, all tumors PIK3CA mutation markers","q-value(%) SAM, all tumors PIK3CA missense mutation in kinase domain markers","q-value(%) SAM, all tumors PIK3CA missense mutation in helical domain markers"
0,True,FAM184B,S976s,NP_056503_S976s _1_1_976_976,NP_056503,0,NaN,4.05,5.81,6.41,6.38,7.46,2.17,0.00,99.17,2.28
1,True,TRIM28,S489s,NP_005753_S489s _1_1_489_489,NP_005753,0,NaN,3.82,3.60,3.38,NaN,8.10,0.00,0.00,NaN,0.00
2,True,ARHGEF35,S450s,NP_001003702_S450s _1_1_450_450,NP_001003702,0,NaN,3.68,3.60,3.44,NaN,3.83,0.00,0.00,NaN,6.24
3,True,RPS6KA5,S212s,NP_004746_S212s _1_1_212_212,NP_004746,1,"MAPK1_RPS6KA5,MAPK3_RPS6KA5",3.50,3.62,2.57,NaN,2.94,0.00,0.00,NaN,5.02
4,True,EIF2AK4,S572s,NP_001013725_S572s _1_1_572_572,NP_001013725,1,NaN,3.48,2.82,2.17,NaN,2.36,0.00,0.00,NaN,7.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25856,False,FAM171A1,S525s,NP_001010924_S525s _1_0_524_525,NP_001010924,0,NaN,-3.25,0.71,0.45,0.42,0.42,41.27,4.89,51.83,62.69
25857,False,TRIM2,S424s S428s,NP_001123539_S424sS428s_2_2_424_428,NP_001123539,0,NaN,-3.34,0.49,0.37,0.41,0.34,30.03,4.08,68.27,62.69
25858,False,MAP2,T1649t S1653s,NP_002365_T1649tS1653s_2_2_1649_1653,NP_002365,0,NaN,-3.40,0.40,0.26,0.22,0.35,30.03,4.08,41.85,67.37
25859,False,MAP2,S1155s,NP_002365_S1155s _1_0_1154_1155,NP_002365,0,NaN,-3.41,0.54,0.34,0.44,0.29,33.94,4.08,74.41,60.12


In [80]:
sup18 = pd.read_csv("~/Documents/CPTAC_BC_SupplementaryTable18.csv")
sup18["phospho_site"] =  sup18["Gene Name"] + "_" + sup18["Phosphosite"]
sup18 = sup18[["Gene Name","FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)", "phospho_site"]]
sup18 = sup18.rename(columns = {"FDR<5% PIK3CA mutation markers in all tumors and luminal tumors only (upregulated in mutated tumors)" : "Significant", "Gene Name" : "Gene_Name"})

#sup18 = sup18.replace(r' ', "_", regex = True)
sup18

,Gene_Name,Significant,phospho_site
0,FAM184B,True,FAM184B_S976s_
1,TRIM28,True,TRIM28_S489s_
2,ARHGEF35,True,ARHGEF35_S450s_
3,RPS6KA5,True,RPS6KA5_S212s_
4,EIF2AK4,True,EIF2AK4_S572s_
...,...,...,...
25856,FAM171A1,False,FAM171A1_S525s_
25857,TRIM2,False,TRIM2_S424s_S428s_
25858,MAP2,False,MAP2_T1649t_S1653s_
25859,MAP2,False,MAP2_S1155s_


In [81]:
sup18_T = sup18.loc[sup18["Significant"] == True]
sup18_T

,Gene_Name,Significant,phospho_site
0,FAM184B,True,FAM184B_S976s_
1,TRIM28,True,TRIM28_S489s_
2,ARHGEF35,True,ARHGEF35_S450s_
3,RPS6KA5,True,RPS6KA5_S212s_
4,EIF2AK4,True,EIF2AK4_S572s_
...,...,...,...
57,INTS1,True,INTS1_S13s_
58,RBM10,True,RBM10_S596s_
59,KCTD1,True,KCTD1_S617s_S620s_
60,TAF3,True,TAF3_S199s_


In [92]:
cols_62 = sup18_T.phospho_site.to_list()
len(cols_62)
cols_62
sup18_T_genes = sup18_T.Gene_Name.drop_duplicates()
genes = sup18_T_genes.to_list()
len(genes)

52

In [87]:
#genes = sup18_T.Gene_Name.to_list()
#len(genes)

62

In [108]:
cols_62

['FAM184B_S976s ',
 'TRIM28_S489s ',
 'ARHGEF35_S450s ',
 'RPS6KA5_S212s ',
 'EIF2AK4_S572s ',
 'SRRT_S136s ',
 'BCL9_S441s ',
 'TP53BP1_S899s ',
 'FAM21A_S836s ',
 'MAPT_S726s T731t ',
 'MAPT_S411s S437s ',
 'TSC22D3_S73s ',
 'HEATR3_T340t ',
 'MED26_T426t ',
 'ACIN1_S863s ',
 'HUWE1_S3919s T3924t T3927t ',
 'GTF3C1_S1632s ',
 'ZC3H14_T204t ',
 'KRT8_S475s ',
 'MAPT_S531s ',
 'SPAG7_S202s ',
 'ARID4B_S666s ',
 'METTL3_S43s ',
 'MAPT_T111t ',
 'MAPT_S531s T534t ',
 'MAPT_S733s ',
 'UBXN2B_S66s ',
 'RAVER1_S584s ',
 'RTF1_S697s ',
 'RXRA_T23t ',
 'PHF20L1_T420t ',
 'WWC3_S957s ',
 'WHSC2_T288t ',
 'GTF2F1_T427t ',
 'RPS6KA5_S694s T700t ',
 'UBXN7_S106s ',
 'BCL2L11_S77s ',
 'NOSIP_S36s ',
 'MPG_C61c T67t ',
 'PPP6R2_S796s ',
 'KHDRBS1_S20s ',
 'FIP1L1_T221t ',
 'STAT3_T714t S727s ',
 'RAPH1_Q843q S853s S856s ',
 'NFIA_S280s M285m S287s ',
 'HUWE1_S1368s ',
 'DDX17_S573s ',
 'TSC22D3_S188s ',
 'NFIA_S300s S319s ',
 'KIAA1468_S439s ',
 'ANXA11_S342s ',
 'RAVER1_S660s ',
 'FAM83H_S1024s ',